# The Data Science Process

This is intended to be a follow-up script for my talk entitled [Launching a Data Science Project:  Cleaning is Half the Battle](http://www.catallaxyservices.com/presentations/datascience/).  I will repeat some of the presentation in this notebook, but this notebook is not designed to replace the presentation itself.

Over the course of this notebook, we will analyze Data Professional salaries based on the [2021 Data Professionals Salary Survey](https://www.brentozar.com/archive/2021/01/are-you-underpaid-find-out-in-the-data-professional-salary-survey-results/).  We will use this data as the centerpiece of an implementation of the [Microsoft Team Data Science Process](https://docs.microsoft.com/en-us/azure/machine-learning/team-data-science-process/overview).

The Team Data Science Process has five major steps:
1. Business Understanding
2. Data Processing
3. Modeling
4. Deployment
5. Repeat

## Business Understanding

Before we begin working, we need to understand the problem.  In our scenario, we work for Data Platform Specialists, a company dedicated to providing DBAs and other data platform professionals with valuable market knowledge. We have come into possession of a survey of data professionals and want to build insights that we can share with our client base.  Our higher-ups want us to analyze this survey and see how we can make good use of it.

During initial brainstorming, we might come up with questions like:
* How much money does a DBA make?
* Which category of DBA (junior, mid-level, senior) does this particular work?
* How can we segment the DBAs in our survey?
* Is this many hours per week weird?
* Which option should I choose as a career path? DBA? Data science? BI?

These questions could potentially be interesting, and the specific form of the question will help guide our analysis.  For example, "how much" questions typically lead to regression, whereas "which category" questions are indicative of a classification problem.  Segmentation questions where we do not know the classes beforehand is a classic example of a clustering problem, and the final two questions are anomaly detection and recommendation, respectively.

Narrowing this down with our champion and other stakeholders, we can get to the following question which we will endeavour to answer:

**How much money should we expect a data professional will make?**

This is a vague question that we will want to focus in on later, but at least it gives us a start.

## Data Processing

There are three steps to data processing:  data gathering, data cleansing, and data analysis.

### Data Gathering

In this example, we will stick to just the data professional survey. But if you want to take this further, a few additional data sources could be:

* PPP GDP per capita to normalize salaries across countries.
* A geocoding data set to visualize results on a map.
* Cost of living by ZIP code to normalize salaries within the US.
* Census information to build out data by ZIP code.
* Data from other surveys to add more to the sample.

If you are interested in extending this to include PPP GDP per capita, check out [a blog post I did based on the 2017 survey results](https://36chambers.wordpress.com/2017/01/18/analyzing-dba-salaries/).

### Data Cleansing

Here is where we begin the real work.  In the next snippet, I am going to load a series of libraries in R.  We will use each of them over the course of this notebook.  The `tidyverse` package is a series of incredibly useful libraries in R, and I can't think of doing a data science project in R without it.  The `XLConnect` package lets me read an Excel workbook easily and grab the salary data without much hassle.  The `caret` library provides some helpful tooling for working with data, including splitting out test versus training data, like we'll do below.  The `recipes` package will be useful for normalizing data later, and we will use `data.table` to get a glimpse at some of our uneven data.  Finally, we will use the `randomForest` package to build out a random forest model, as these tend to be a good first guess for regression solutions.

In [ ]:
if(!require(tidyverse)) {
  install.packages("tidyverse", repos = "http://cran.us.r-project.org")
  library(tidyverse)
}

if(!require(caret)) {
  install.packages("caret", repos = "http://cran.us.r-project.org")
  library(caret)
}

if(!require(recipes)) {
  install.packages("recipes", repos = "http://cran.us.r-project.org")
  library(recipes)
}

if(!require(data.table)) {
  install.packages("data.table", repos = "http://cran.us.r-project.org")
  library(data.table)
}

if(!require(randomForest)) {
  install.packages("randomForest", repos = "http://cran.us.r-project.org")
  library(randomForest)
}

Once we have the required packages loaded, we will then load the Excel workbook.  I have verified the Excel worksheet and data region are correct, so we can grab the survey from the current directory and load it into `salary_data`.

In [ ]:
salary_data <- readxl::read_excel("2021_Data_Professional_Salary_Survey_Responses.xlsx", skip=3, sheet="Salary Survey", guess_max = 6000)

We can use the `glimpse` function inside the tidyverse to get a quick idea of what our `salary_data` dataframe looks like.  In total, we have 10,341 observations of 31 variables, but this covers five survey years:  2017 through 2021.  Looking at the variable names, we can see that there are some which don't matter very much (like Timestamp, which is when the user filled out the form; and Counter, which is just a 1 for each record.

In [ ]:
glimpse(salary_data)

Our first data cleansing decision point is whether to remove certain years of data.  There are new questions in each year of the data, so we have a choice:  either ignore the questions which were asked later or remove the years of data which did not ask the relevant questions.

The most important question in 2018 was Gender, which had a noticeable impact on salaries, as you can see from the notebook which covers the 2018 salary survey.  In this survey, I will choose to keep all three years and will drop the new questions.  This lets me use all 6893 responses over the past three years.

Note that I do not need to create a new variable here, but it is usually a good idea to do so if you can afford the memory utilization, as that means you can retain an original view of the data just in case things go sideways during cleansing and analysis.

In [ ]:
survey_data <- salary_data
nrow(survey_data)

Looking at the survey, there are some interesting data points that we want:
* SalaryUSD (our label, that is, what we are going to try to predict)
* Country
* PrimaryDatabase
* YearsWithThisDatabase
* EmploymentStatus
* JobTitle
* ManageStaff
* YearsWithThisTypeOfJob
* OtherPeopleOnYourTeam
* DatabaseServers
* Education
* EducationIsComputerRelated
* Certifications
* HoursWorkedPerWeek
* TelecommuteDaysPerWeek
* EmploymentSector
* LookingForAnotherJob

For each of these variables, we want to see the range of options and perform any necessary cleanup.  The first thing I'd look at is the cardinality of each variable, followed by a detailed anlaysis of the smaller ones.

In [ ]:
rapply(survey_data, function(x) { length(unique(x)) })

In [ ]:
unique(survey_data$Country)

In [ ]:
unique(survey_data$EmploymentStatus)

We can use the `setDT` function on `data.table` to see just how many records we have for each level of a particular factor.  For example, we can see the different entries for PrimaryDatabase and EmploymentSector below.  Both of these are troublesome for our modeling because they both have a number of levels with 1-2 entries.  This makes it likely that we will fail to collect a relevant record in our training data set, and that will mess up our model later.  

PrimaryDatabase is also troublesome because it skews so heavily toward SQL Server that there's more noise than signal to it.  Because there are so many platforms with 10 or fewer entries and about 94% of entrants selected SQL Server, we'll throw it out.

In [ ]:
data.table::setDT(survey_data)[, .N, keyby=PrimaryDatabase]

There are only nine students in this survey.  That's not enough data points to draw a meaningful conclusion, so I will throw those nine out.

In [ ]:
data.table::setDT(survey_data)[, .N, keyby=EmploymentSector]

Most of these columns came from dropdown lists, so they're already fairly clean.  But there are some exceptions to the rule.  They are:
* SalaryUSD
* YearsWithThisDatabase
* YearsWithThisTypeOfJob
* DatabaseServers
* HoursWorkedPerWeek

All of these were text fields, and whenever a user gets to enter text, you can assume that something will go wrong.  For example:

In [ ]:
survey_data %>%
  distinct(YearsWithThisDatabase) %>%
  arrange(desc(YearsWithThisDatabase)) %>%
  top_n(10, YearsWithThisDatabase)

Someone with 53,716 years working with their primary database of choice?  That's commitment!  You can also see a couple of people who clearly put in the year they started rather than the number of years working with it, and someone who maybe meant 10 years?  But who knows, people type in weird stuff.

Anyhow, let's see how much those people with at least 10 thousand years of experience make:

In [ ]:
survey_data %>%
  filter(YearsWithThisDatabase > 10000)

That's pretty sad, considering their millennia of work experience.  $95-98K isn't even that great a number.

Looking at years of experience with their current job roles, looks like we still have a couple of problems, though for the most part the high end is reasonable:

In [ ]:
survey_data %>%
  distinct(YearsWithThisTypeOfJob) %>%
  arrange(desc(YearsWithThisTypeOfJob)) %>%
  top_n(10, YearsWithThisTypeOfJob)

Now let's look at hours per week:

In [ ]:
survey_data %>%
  distinct(HoursWorkedPerWeek) %>%
  arrange(desc(HoursWorkedPerWeek)) %>%
  top_n(10, HoursWorkedPerWeek)

To the person who works 200 hours per week:  find a new job.  Your ability to pack more than 7\*24 hours of work into 7 days is too good to waste on a job making just $120K per year.

In [ ]:
survey_data %>%
  filter(HoursWorkedPerWeek >= 168) %>%
  arrange(desc(HoursWorkedPerWeek))

There are only twenty-four countries with at least 30 data points.  This means that we won't get a great amount of information from cross-country comparisons outside of the sample.  Frankly, I might want to limit this to just the US, UK, Canada, and maybe Australia, as the rest don't have enough entries to make me really comfortable.  But for this survey analysis, I'll keep the remainder.

In [ ]:
survey_data %>%
  group_by(Country) %>%
  summarize(n = n()) %>%
  filter(n >= 30)

#### Building Our Cleaned-Up Data Set

Now that we've performed some basic analysis, we will clean up the data set.  I'm doing most of the cleanup in a single operation, but I do have some comment notes here, particularly around the oddities with SalaryUSD.  The SalaryUSD column has a few problems:
* Some people put in pennies, which aren't really that important at the level we're discussing.  I want to strip them out.
* Some people put in delimiters like commas or decimal points (which act as commas in countries like Germany).  I want to strip them out, particularly because the decimal point might interfere with my analysis, turning 100.000 to \\$100 instead of \\$100K.
* Some people included the dollar sign, so remove that, as well as any spaces.

It's not a perfect regex, but it did seem to fix the problems in this data set at least.

In [ ]:
valid_countries <- survey_data %>%
                    group_by(Country) %>%
                    summarize(n = n()) %>%
                    filter(n >= 30)

# Data cleanup
survey_data <- salary_data %>%
  filter(HoursWorkedPerWeek < 200) %>%
  # There were only nine students in the survey, so we will exclude them here.
  filter(EmploymentSector != "Student") %>%
  inner_join(valid_countries, by="Country") %>%
  mutate(
    SalaryUSD = stringr::str_replace_all(SalaryUSD, "\\$", "") %>%
      stringr::str_replace_all(., ",", "") %>%
      stringr::str_replace_all(., " ", "") %>%
      # Some people put in pennies.  Let's remove anything with a decimal point and then two numbers.
      stringr::str_replace_all(., stringr::regex("\\.[0-9]{2}$"), "") %>%
      # Now any decimal points remaining are formatting characters.
      stringr::str_replace_all(., "\\.", "") %>%
      as.numeric(.),
    # Some people have entered bad values here, so set them to the median.
    YearsWithThisTypeOfJob = case_when(
        (YearsWithThisTypeOfJob > 60) ~ median(YearsWithThisTypeOfJob),
        TRUE ~ YearsWithThisTypeOfJob
    ),
    YearsWithThisDatabase = case_when(
      (YearsWithThisDatabase > 32) ~ median(YearsWithThisDatabase),
      TRUE ~ YearsWithThisDatabase
    ),
    # Some people apparently entered number of databases rather than number of servers.
    DatabaseServers = case_when(
      (DatabaseServers >= 5000) ~ median(DatabaseServers),
      TRUE ~ DatabaseServers
    ),
    EmploymentStatus = as.factor(EmploymentStatus),
    JobTitle = as.factor(JobTitle),
    ManageStaff = as.factor(ManageStaff),
    OtherPeopleOnYourTeam = as.factor(OtherPeopleOnYourTeam),
    Education = as.factor(Education),
    EducationIsComputerRelated = as.factor(EducationIsComputerRelated),
    Certifications = as.factor(Certifications),
    TelecommuteDaysPerWeek = as.factor(TelecommuteDaysPerWeek),
    EmploymentSector = as.factor(EmploymentSector),
    LookingForAnotherJob = as.factor(LookingForAnotherJob)
  ) 

Now we can pare out variables we don't need.  In many cases, this is information which was not asked in 2017, so we choose to elimintate them to keep the ~3000 survey responses in place.

Some of these eliminated columns, like postal code, are interesting but we just don't have enough data for it to make sense.  Others, like Kinds of Tasks Performed or Other Job Duties, have too many varieties for us to make much sense with a first pass.  They might be interesting in a subsequent analysis, though.

In [ ]:
survey_data <- survey_data %>%
  # One person had a salary of zero.  That's just not right.
  filter(SalaryUSD > 0) %>%
  select(-`Survey Year`, -Timestamp, -PostalCode, -PrimaryDatabase, -OtherDatabases, -HowManyCompanies,
         -CompanyEmployeesOverall, -PopulationOfLargestCityWithin20Miles, -CareerPlansThisYear, -Gender,
         -OtherJobDuties, -KindsOfTasksPerformed, -NewestVersionInProduction, -OldestVersionInProduction, -Counter, -n)

Now that we have our salary data fixed, we can finally look at outliers.  I'd consider a salary of \\$500K a year to be a bit weird for this field.  It's certainly not impossible, but it is interesting.  I am very suspicious of the part-timer making \\$1.375 million, the federal employee making \\$1 million, or the New Zealander making \\$630K at a non-profit.

Because these outcomes are rare and extreme, I'm not sure we can predict them with any accuracy given our information, so let's leave them out with the caveat that our model will suffer in those rare cases.

In [ ]:
survey_data %>%
  filter(SalaryUSD > 500000) %>%
  arrange(desc(SalaryUSD))

On the other side, there are a fair number of people who say they earned less than $5K a year.  Those also seem wrong.  Some of them look like dollars per hour at the low end, and maybe some are semi-weekly or monthly salary.  I'm going to strip those out.

In [ ]:
survey_data %>%
  filter(SalaryUSD < 3000) %>%
  arrange(desc(SalaryUSD))

In [ ]:
survey_data <- filter(survey_data, SalaryUSD >= 3000 & SalaryUSD <= 500000)

### Data Analysis

We did some of the data analysis up above.  We can do additional visualization and correlation studies.  For example, let's look at a quick distribution of salaries after our cleanup work:

In [ ]:
summary(survey_data$SalaryUSD)

We can also build a histogram pretty easily using the `ggplot2` library.  This shows the big clump of database professionals earning beween \\$70K and \\$115K per year.  This salary distribution does skew right a bit, as you can see.

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)
ggplot(data = survey_data, mapping = aes(x = SalaryUSD)) +
  geom_histogram() +
  theme_minimal() +
  scale_x_log10(label = scales::dollar)

We can also break this down to look by primary job title, though I'll limit to a couple of summaries instead of showing a full picture.

In [ ]:
survey_data %>% filter(JobTitle == "Data Scientist") %>% select(SalaryUSD) %>% summary(.)

In [ ]:
survey_data %>% filter(JobTitle == "Developer: App code (C#, JS, etc)") %>% select(SalaryUSD) %>% summary(.)

In [ ]:
survey_data %>% filter(JobTitle == "Developer: T-SQL") %>% select(SalaryUSD) %>% summary(.)

This fit pretty well to my biases, with Data Scientist salaries perhaps a bit lower than I'd expect and we have one crazy outlier for T-SQL developers.

## Modeling

Because our question is a "how much?" question, we want to use regression to solve the problem.  The most common form of regression that you'll see in demonstrations is linear regression, because it is easy to teach and easy to understand.  In today's demo, however, we're going to build a neural network with Keras.  Although our demo is in R, Keras actually uses Python on the back end to run TensorFlow.  There are other libraries out there which can run neural networks strictly within R (for example, Microsoft Machine Learning's R implemenation has the `RxNeuralNet()` function), but we will use Keras in this demo because it is a popular library.

Now that we have an algorithm and implementation in mind, let's split the data out into training and test subsets.  I want to use Country as the partition variable because I want to ensure that we retain some data from each country in the test set.  To make this split, I am using the `createDataPartition()` function in `caret`.  I'll then split out the data into training and test data.

In [ ]:
set.seed(20191119)
rand_survey_data <- survey_data[sample(nrow(survey_data)), ]
trainIndex <- caret::createDataPartition(survey_data$Country, p = 0.7, list = FALSE, times = 1)
train_data <- survey_data[trainIndex,]
test_data <- survey_data[-trainIndex,]
nrow(train_data)
nrow(test_data)

Once I have this data split, I want to perform some operations on the training data.  Specifically, I want to do the following:
* One-Hot Encode the categorical data
* Mean-center the data, so that the mean of each numeric value is 0
* Scale the data, so that the standard deviation of each value is 1

The bottom two are called **normalizing** the data.  This is a valuable technique when dealing with many algorithms, including neural networks, as it helps with optimizing gradient descent problems.

In order to perform all of these operations, I will create a `recipe`, using the `recipes` package.

**NOTE:**  It turns out that normalizing the features results in a slightly worse outcome in this case, so I'm actually going to avoid that.  You can uncomment the two sections and run it yourself if you want to try.  In some problems, normalization is the right answer; in others, it's better without normalization.

In [ ]:
rec_obj <- recipes::recipe(SalaryUSD ~ ., data = train_data) %>%       # Build out a set of rules we want to follow (a recipe)
  step_dummy(all_nominal(), -all_outcomes()) %>%              # One-hot encode categorical data
  #step_center(all_predictors(), -all_outcomes()) %>%          # Mean-center the data
  #step_scale(all_predictors(), -all_outcomes()) %>%           # Scale the data
  prep(data = train_data)

rec_obj

Now we can `bake` our data based on the recipe above.  Note that I performed all of these operations only on the **training** data.  If we normalize the training + test data, our optimization function can get a sneak peek at the distribution of the test data based on what is in the training set, and that will bias our result.

After building up the `x_` series of data sets, I'll build vectors which contain the salaries for the training and test data.  I need to make sure to remove the SalaryUSD variable; we don't want to make that available to the trainer as an independent variable!

In [ ]:
x_train_data <- recipes::bake(rec_obj, new_data = train_data)
x_test_data <- recipes::bake(rec_obj, new_data = test_data)

At this point, I want to build the random forest model.  In a real-life scenario, I might perform various optimizations, do cross-validation, etc.  In this scenario, however, I am just going to run one time against the full training data set, and then evaluate it against the test data set.

Note that this will probably take a while to build, as we have a lot of data and quite a few trees to create.

In [ ]:
model <- randomForest::randomForest(SalaryUSD ~ .,
               data = x_train_data,
               ntree=400,
               importance=TRUE
           )

Now that we have a trained model, let's see which features are most important for determining `SalaryUSD`.  As a programming note, any value with `%IncMSE` less than 0 is not important, and the larger the number, the more important this value is for prediction.  This is **not** a listing of weights, though!  It just measures how important the value is to determining a weight.

In [ ]:
randomForest::importance(model, scale=TRUE)

Now we want to generate predictions against our test data.

In [ ]:
modelPred <- predict(model, x_test_data)

Let's combine the predictions with our test data.  Here, I'm column-binding the model predictions with the original test data rather than the one-hot encoded version, as that means many fewer columns.

In [ ]:
outcomes <- cbind(as.data.frame(modelPred), test_data)

Taking a brief look at this, the first few rows don't look so bad.  But that's not a comprehensive understanding of how we're doing.

In [ ]:
head(outcomes)

Root Mean Square Error (RMSE) gives us a calculation of how far off we are.  The nice thing about this calculation is that the number will be in US Dollars, the unit of our analysis.

In [ ]:
RMSE = function(m, o){
  sqrt(mean((m - o)^2))
}

RMSE(outcomes$SalaryUSD, outcomes$modelPred)

What this tells us is that, after generating our model, we are an average of about \\$28K off.  That's probably not good enough to roll out to production with, but with a few other enhancements, we might see that number drop a bit and start getting into the trustworthy territory.

With a real data science project, I would dig further, seeing if there were better algorithms available, cross-validating the training set, etc.  As-is, this result isn't good enough for a production scenario, but we can pretend that it is.

## Quick Validation And Testing

Now let's test scenarios.  I will use three actual times in my career:  when I started as a SQL Server developer, when I was ready to move on from a job in 2013, and when I became a manager of a predictive analytics team.  As a twist, we'll also include the third scenario, but in a hypothetical world where I moved to Canada.

In [ ]:
test_cases <- test_data[1:4, ]

test_cases$SalaryUSD = c(1,2,3,4)
test_cases$Country = c("United States", "United States", "United States", "Canada")
test_cases$YearsWithThisDatabase = c(0, 5, 11, 11)
test_cases$EmploymentStatus = c("Full time employee", "Full time employee", "Full time employee", "Full time employee")
test_cases$JobTitle = c("Developer: App code (C#, JS, etc)", "DBA (General - splits time evenly between writing & tuning queries AND building & troubleshooting servers)", "Manager", "Manager")
test_cases$ManageStaff = c("No", "No", "Yes", "Yes")
test_cases$YearsWithThisTypeOfJob = c(0, 5, 0, 0)
test_cases$OtherPeopleOnYourTeam = c("5", "None", "2", "2")
test_cases$DatabaseServers = c(8, 12, 150, 150)
test_cases$Education = c("Bachelors (4 years)", "Masters", "Masters", "Masters")
test_cases$EducationIsComputerRelated = c("Yes", "Yes", "Yes", "Yes")
test_cases$Certifications = c("No, I never have", "Yes, and they're currently valid", "Yes, but they expired", "Yes, but they expired")
test_cases$HoursWorkedPerWeek = c(40, 40, 40, 40)
test_cases$TelecommuteDaysPerWeek = c("None, or less than 1 day per week", "None, or less than 1 day per week", "None, or less than 1 day per week", "None, or less than 1 day per week")
test_cases$EmploymentSector = c("State/province government", "State/province government", "Private business", "Private business")
test_cases$LookingForAnotherJob = c("No", "Yes, actively looking for something else", "No", "No")

In [ ]:
x_test_cases <- recipes::bake(rec_obj, new_data = test_cases)
x_test_cases <- x_test_cases %>% select(-SalaryUSD)

In [ ]:
model %>% predict(as.matrix(x_test_cases))

The first and third predictions were somewhere in the ballpark, though none was spot on.  The second prediction was quite a bit off.  The Canadian rate is interesting, as \\$1 Canadian Dollar has floated between \\$0.7 and \\$0.8 USD, so that might be a factor in play, but there's definitely more to it than that.

## Deployment

During the deployment phase, we will need to take our web service's inputs and transform them into the same shape as our training and test data.  We would then run the set and output our SalaryUSD prediction as the result.

I'm leaving the details of that as an exercise (because there are so many variables around deployment, including how you will fit it into your normal infrastructure), but a few thoughts:

1. As mentioned in the scenario, we're going to run a lot of one-off predictions rather than taking a batch of inputs, we want to optimize this for speed.  We could probably make this an asynchronous call so that when one of our users changes a detail like number of years working at the job, we can update our salary estimate.
2. This would probably be a good use case for SQL Server Machine Learning Services:  if we save all of the profile data in the database, we can pass those results straight to ML Services and perform the transformation and prediction in R.  If we go this route, we'd return the results as a SQL Server result set, which is easy for most developers to handle.
3. Another option would be to rewrite this in Python, as we're using Python under the covers anyhow.  Then we could use a web service framework like Flask to build a simple API.  If you go that route, you might want to stand up a production-grade service like Gunicorn and use Nginx as the front door to handle traffic and load balancing.

## Repeat The Process

From here, we would collect data on actual predictions.  We saw the test set's Mean Absolute Error, but an even more valuable measure is the Mean Absolute Error of actual data.  We'd need to ask people what their actual salaries are to know for sure, but we could feed these back into the training data and keep training our model.  One of the nice things about using a neural network like this is that we can continuously train the model based on our newly-gathered data.

Next, we would want to follow up with our stakeholders.  If they're happy with this result, we could probably look at other uses for the data source, such as cleaning up and including postal code, or integrating with a service to get more granular information on salary in certain parts of larger countries like the US and Canada.